# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [38]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [39]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [40]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 


# for every filepath in the file path list 
for f in file_path_list:
    
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile)
        next(csvreader) #--> to past the headers
        
        # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            

# get the total number of rows 
print(len(full_data_rows_list))

# check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will 
# be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], \
                         row[12], row[13], row[16]))


8056


In [41]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("total number of lines :", sum(1 for line in f))

total number of lines : 6821


### Check for nulls 

In [42]:
# convert data into a pandas dataframe in order to check for nulls 
songs_df = pd.read_csv('event_datafile_new.csv')
songs_df.isnull().values.any()


False

#### Check for clean data

In [43]:
# get the csv file into a pandas dataframe
event_df = pd.read_csv('event_datafile_new.csv')

n_df = event_df[event_df.artist.str.contains('/')]
n_df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
15,Mos Def / Talib Kweli,Aleena,F,8,Kirby,141.37424,paid,"Waterloo-Cedar Falls, IA",350,History,44


#### Analyzing the csv file just in case it might need some cleaning


In [44]:
event_df.describe()
event_df.shape

(6820, 11)

In [45]:
## eliminate duplicates
event_df.drop_duplicates(inplace=True)
event_df.shape

(6820, 11)

In [46]:
event_df[event_df['artist'].str.contains('/')].any()

artist           True
firstName        True
gender           True
itemInSession    True
lastName         True
length           True
level            True
location         True
sessionId        True
song             True
userId           True
dtype: bool

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [47]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [48]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS songslib 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [49]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('songslib')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4¶

In [50]:
## FIRST --> create the table song_detail where primary key is the combination of session_id 
# and item_in_session.
# partition key is session_id and item_in_session is the clustering key

# drop table if it exists
query = "DROP TABLE IF EXISTS song_detail;"
try:
    session.execute(query)
except Exception as e:
    print(e)

# next create the table
query = """CREATE TABLE IF NOT EXISTS song_detail
        (session_id int, item_in_session int, artist text, song_title text, song_length float, 
        PRIMARY KEY (session_id, item_in_session));"""
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [51]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_detail (session_id, item_in_session, artist, \
            song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)    
                            

#### Do a SELECT to verify that the data have been inserted into each table

In [52]:
## Get the count to verify the data was entered into the table

que = """SELECT COUNT(*) FROM song_detail;"""
try:
    rows = session.execute(que)
except Exception as e:
    print(e)
    
print("total number of rows :", rows[0][0] )


total number of rows : 6820


In [53]:
## Get the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

qu = """SELECT artist, song_title, song_length 
        FROM song_detail
        WHERE session_id=%s AND item_in_session=%s """
try:
    rows = session.execute(qu, (338, 4))
except Exception as e:
    print(e)

for row in rows:
    print("Artist Name:", row.artist, ", Song Title: ",  row.song_title, ", Duration :", row.song_length)   

Artist Name: Faithless , Song Title:  Music Matters (Mark Knight Dub) , Duration : 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### I made an analysis on the data and I found out that in order to have rows  uniqeness  I am incuding item_in_session as well. Primary key is the combination of  (user_id, session_id) as partition_key and item_in_session as the  clustering key

In [54]:
# Create the table artist_detail 
# first drop it in case it exists 

query = "DROP TABLE IF EXISTS artist_detail;"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
#Then go ahead and create the table
query = """CREATE TABLE IF NOT EXISTS artist_detail 
        (user_id int, session_id int, item_in_session int, 
        artist text, song_title text, user_first text, user_last text, 
        PRIMARY KEY ((user_id, session_id), item_in_session));"""
        #WITH CLUSTERING ORDER BY (user_id ASC, session_id ASC, item_in_session ASC));"""
try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [55]:
# insert data into artist_detail 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = """INSERT INTO artist_detail (user_id, session_id, item_in_session, artist, 
                song_title, user_first, user_last)"""
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], 
                                    line[1], line[4]))
        except Exception as e:
            print(e)    


In [83]:
qur = """SELECT count(*) FROM artist_detail;"""
try:    
    rows = session.execute(qur)
    
except Exception as e:
    print(e)
    
for row in rows:
    print(row[0])

6820


In [56]:
# Before querying the data insertion I am converting the query into a pandas dataframe 
# in order to analyze it using Python
qu = """SELECT artist, song_title, user_first, user_last 
            FROM artist_detail;"""
try:
    rows = session.execute(qu)
except Exception as e:
    print(e)
    
artist_df = pd.DataFrame(columns = ['artist', 'song_title', 'user_first', 'user_last'])

for row in rows:
    artist_df.loc[len(artist_df)] = [row.artist, row.song_title, row.user_first, row.user_last ]

artist_df.head()


,artist,song_title,user_first,user_last
0,System of a Down,Sad Statue,Emily,Benson
1,Ghostland Observatory,Stranger Lover,Emily,Benson
2,Evergreen Terrace,Zero,Emily,Benson
3,Deftones,Head Up (LP Version),Kinsley,Young
4,The Notorious B.I.G.,Playa Hater (Amended Version),Kinsley,Young


In [59]:
artist_df.duplicated().head()

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [60]:
# And now the selection
qu = """SELECT artist, song_title, user_first, user_last 
        FROM artist_detail 
        WHERE user_id=%s
        AND session_id=%s 
        ORDER BY item_in_session;"""
        
try:    
    rows = session.execute(qu, (10, 182))

except Exception as e:
    print(e)

for row in rows:
    print("Artist :", row.artist, ", Song: ",  row.song_title, ", First :", row.user_first, "Last:" , row.user_last)      

Artist : Down To The Bone , Song:  Keep On Keepin' On , First : Sylvie Last: Cruz
Artist : Three Drives , Song:  Greece 2000 , First : Sylvie Last: Cruz
Artist : Sebastien Tellier , Song:  Kilometer , First : Sylvie Last: Cruz
Artist : Lonnie Gordon , Song:  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , First : Sylvie Last: Cruz


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [118]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "DROP TABLE IF EXISTS song_stats;"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
#Then go ahead and create the table
query = """CREATE TABLE IF NOT EXISTS song_stats 
        (song text, user_id int, user_last text, user_first text, 
        PRIMARY KEY (song, user_id));"""
       
try:
    session.execute(query)
except Exception as e:
    print(e)                        

In [120]:
# insert data into song_stats table 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = """INSERT INTO song_stats (song, user_id, user_last, user_first)"""
        query = query + " VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], int(line[10]), line[4], line[1]))
        except Exception as e:
            print(e)

In [121]:
qur = """SELECT count(*) FROM song_stats;"""
try:    
    rows = session.execute(qur)
    
except Exception as e:
    print(e)
    
for row in rows:
    print(row[0])

6618


In [98]:
## check for existennce on the csv file
event_df[event_df.song.str.contains('All Hands Against His Own')].head()


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1685,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
6301,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6618,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


In [66]:
event_df[event_df.song == 'All Hands Against His Own'].head()


,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
1685,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80
6301,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
6618,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29


In [122]:
# let's check the new table --> I am converting the query into a pandas dataframe 
# in order to analyze it using Python
qu = """SELECT song, user_id, user_last, user_first FROM song_stats;"""
try:
    rows = session.execute(qu)
except Exception as e:
    print(e)
    
songs_df = pd.DataFrame(columns = ['song', 'user_id', 'user_last', 'user_first'])

for row in rows:
    songs_df.loc[len(songs_df)] = [row.song, row.user_id, row.user_last , row.user_first ]

songs_df.shape   


(6618, 4)

In [123]:
songs_df[songs_df.song == 'All Hands Against His Own'].head()

,song,user_id,user_last,user_first
4008,All Hands Against His Own,29,Lynch,Jacqueline
4009,All Hands Against His Own,80,Levine,Tegan
4010,All Hands Against His Own,95,Johnson,Sara


In [124]:
songs_df[songs_df['song'] == 'All Hands Against His Own'].head()

,song,user_id,user_last,user_first
4008,All Hands Against His Own,29,Lynch,Jacqueline
4009,All Hands Against His Own,80,Levine,Tegan
4010,All Hands Against His Own,95,Johnson,Sara


In [125]:
songs_df.shape

(6618, 4)

In [142]:
## actual query
#qur = """SELECT song, user_last, user_first FROM song_stats WHERE song = 'All Hands Against His Own';"""#
#print(qur)

search_song = 'All Hands Against His Own'
qury = "SELECT song, user_last, user_first FROM song_stats WHERE song = '" + search_song + "';"

try:    
    rows = session.execute(qury)
    
except Exception as e:
    print(e)
    
for row in rows:
    print("Song :", row.song, ", User Lastname: ",  row.user_last, ", User FirstName :", row.user_first)      

Song : All Hands Against His Own , User Lastname:  Lynch , User FirstName : Jacqueline
Song : All Hands Against His Own , User Lastname:  Levine , User FirstName : Tegan
Song : All Hands Against His Own , User Lastname:  Johnson , User FirstName : Sara


### Drop the tables before closing out the sessions

In [32]:
## drop table song_detail
qu = "DROP TABLE IF EXISTS song_detail;"
try:
    rows = session.execute(qu)
except Exception as e:
    print(e)
    
    
## drop table artist_detail
qu = "DROP TABLE IF EXISTS artist_detail;"

try:
    rows = session.execute(qu)
except Exception as e:
    print(e)
    

## drop table song_stats
qu = "DROP TABLE IF EXISTS song_stats;"
try:
    rows = session.execute(qu)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()